###  Data Preparation and Feature Extraction

In [ ]:
import os
import matplotlib.pyplot as plt
import numpy as np
import librosa
import glob

# Saving the path of all training data in one list
data = glob.glob("train\\1piano\\*")
data.extend(glob.glob("train\\2guitar\\*"))
data.extend(glob.glob("train\\3violin\\*"))
data.extend(glob.glob("train\\4drum\\*"))

X = []
Y = []

# Create a directory to save MFCC plots
output_dir = "mfcc_plots"
os.makedirs(output_dir, exist_ok=True)

# Extracting the features using MFCC
for file in data:
    audio, rate = librosa.load(file, mono=True, duration=30)
    mfccs = librosa.feature.mfcc(y=audio, sr=rate, n_mfcc=15)
    mfccs_padded = np.pad(mfccs, pad_width=((0, 0), (0, 500))) # Pad MFCCs for consistent dimensions  
    label = int(file.split("$")[1][0])
    
    X.append(np.mean(mfccs_padded, axis=1)) # Taking the mean to reduce the feature dimensionality 
    Y.append(label)
    
    # Save MFCC plot (commented out for now)
    #plt.figure(figsize=(10, 4))
    #librosa.display.specshow(mfccs, x_axis='time', cmap='Greens')
    #plt.colorbar()
    #plt.title(f'MFCCs for {file}')
    #plt.savefig(os.path.join(output_dir, f'mfcc_plot_{label}_{os.path.basename(file)}.png'))
    #plt.close()
    
# Convert lists to NumPy arrays
X = np.array(X)
Y = np.array(Y)


### Data Splitting for Training/Validation Sets

In [ ]:
from sklearn.model_selection import train_test_split

#Dividing the data into 80% training 20% test
X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size=0.20)

### Input Reshaping for Model Compatibility

In [ ]:
# Reshaping the input to ensure compatibility

X_train = X_train.reshape(X_train.shape[0], X_train.shape[1], 1)
X_test = X_test.reshape(X_test.shape[0], X_test.shape[1], 1)

### Label Encoding and One-Hot Encoding

In [ ]:
import tensorflow as tf
from sklearn.preprocessing import LabelEncoder

# Encode labels
label_encoder = LabelEncoder()
y_train_encoded = label_encoder.fit_transform(y_train)
y_test_encoded = label_encoder.transform(y_test)
y_train_onehot = tf.keras.utils.to_categorical(y_train_encoded, num_classes=4)
y_test_onehot = tf.keras.utils.to_categorical(y_test_encoded, num_classes=4)


### Convolutional Neural Network (CNN) Model Training

In [ ]:
from tensorflow.keras import models, layers


model = models.Sequential()
model.add(layers.Conv1D(32, 3, activation='relu', input_shape=(X_train.shape[1], 1))) # First convolutional layer with 32 filters
model.add(layers.BatchNormalization()) #Batch normalization to improve training stability
model.add(layers.MaxPooling1D(2))

model.add(layers.Conv1D(64, 3, activation='relu')) # Second convolutional layer with 64 filters
model.add(layers.BatchNormalization())
model.add(layers.MaxPooling1D(2))

model.add(layers.Flatten())
model.add(layers.Dense(128, activation='relu')) 
model.add(layers.Dropout(0.5))
model.add(layers.Dense(4, activation='softmax')) # Fully connected output layer with 4 neurons and softmax activation

model.compile(optimizer='adam',
              loss='categorical_crossentropy',
              metrics=['accuracy'])

# Training the model using 40 epochs
epochs = 40
trained_model = model.fit(X_train, y_train_onehot, epochs=epochs, validation_data=(X_test, y_test_onehot))

# Evaluate and print the validation accuracy
accuracy = model.evaluate(X_test, y_test_onehot)[1]
print(f"Validation Accuracy: {accuracy}")

### Plotting Training and Validation Curves

In [ ]:
# Plot training and validation loss curves
plt.figure(figsize=(12, 6))

# Plot training & validation loss values
plt.subplot(1, 2, 1)
plt.plot(trained_model.history['loss'])
plt.plot(trained_model.history['val_loss'])
plt.title('Model Loss')
plt.xlabel('Epoch')
plt.ylabel('Loss')
plt.legend(['Train', 'Validation'], loc='upper right')

# Plot training & validation accuracy values
plt.subplot(1, 2, 2)
plt.plot(trained_model.history['accuracy'])
plt.plot(trained_model.history['val_accuracy'])
plt.title('Model Accuracy')
plt.xlabel('Epoch')
plt.ylabel('Accuracy')
plt.legend(['Train', 'Validation'], loc='lower right')

# Save the plots
plt.savefig('training_validation_curvesAT30.png')
plt.show()

### Model Evaluation and Confusion Matrix Visualization

In [ ]:
from sklearn.metrics import confusion_matrix, ConfusionMatrixDisplay

y_pred = model.predict(X_test)
y_pred = np.argmax(y_pred, axis=1)
cm = confusion_matrix(np.argmax(y_test_onehot, axis=1), y_pred)

# Plot confusion matrix
plt.figure(figsize=(8, 8))
disp = ConfusionMatrixDisplay(confusion_matrix=cm, display_labels=['Piano', 'Guitar', 'Violin', 'Drum'])
disp.plot(cmap=plt.cm.Greens, values_format=".0f")
plt.title('Confusion Matrix')
plt.savefig('confusion_matrix.png')
plt.show()

### Model Summary and Saving

In [ ]:
# Saving the model
model.summary()
model.save("Musical_Instruments_Detection_CNNModel.keras")